In [ ]:
#Imports and Constants
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import sqlalchemy as sql
import itertools
import glob
from openpyxl import load_workbook

#Set these values for each run
#Dieser Wert bestimmt der Auftrag, der vervollständigt werden soll.
AuftragPath = './absys_files/A-2017-0060_919B.XLSX'
#Dieser Wert bestimmt die ExcelDatei in der die Heizkörper bewertet wurden.
HeizkoerperfaktordatenPath = './absys_files/HK_Gesobau_Steffen.xlsx'
#Diese Regular expression bestimmt die Protokolle, aus denen die Daten ausgelesen werden sollen.
protoPathPattern='./absys_files/SenftenbergerRing*'

In [ ]:
#Functions
def findZählernummer(ZNr, df):
    if isinstance(int(ZNr), int):
        assert len(str(ZNr)) == 8
        return df[(df['ZählernummerAlt'] == ZNr)]

def protoCleanupHeizung(df):
    df2append = df.dropna(subset=['Baulänge','Bauhöhe','Bautiefe','HK-Art'])
    df2append = df2append[(df2append['Baulänge'] != 'Baulänge') &
                          (df2append['Bauhöhe'] != 'Bauhöhe')&
                          (df2append['Bautiefe'] != 'Bautiefe') &
                          (df2append['ZählernummerAlt'] != 'Altgerät') &
                          (df2append['ZählernummerNeu'] != 'Neugerät')]
    df2append = df2append[(df2append['Baulänge'] != 'D') &
                          (df2append['Bauhöhe'] != 'E') &
                          (df2append['Bautiefe'] != 'F') &
                          (df2append['ZählernummerAlt'] != 'Nr.') &
                          (df2append['ZählernummerNeu'] != 'Nr.')]
    return df2append

def protoCleanupWasser(df):
    df2append = df.dropna()
    df2append = df2append[(df2append['ZählernummerAlt'] != 'Gerätenummer') &
                          (df2append['ZählerStandAlt'] != 'Stand in m³')&
                          (df2append['ZählernummerNeu'] != 'Gerätenummer') &
                          (df2append['ZählerStandNeu'] != 'Stand in m³')]
    df2append = df2append[(df2append['ZählernummerAlt'] != 'E') &
                          (df2append['ZählerStandAlt'] != 'J') &
                          (df2append['ZählernummerNeu'] != 'Nr.') &
                          (df2append['ZählerStandNeu'] != 'Mont. Pkt.')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Bowa')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Veha')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Fix')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Kermi')]
    return df2append

def collectProtocols(pathPattern):
    #Hier werden die Heizkostenverteilerdaten eingelesen
    protoColumns = {'HK-Art':str,
                    'Baulänge':str,
                    'Bauhöhe':str,
                    'Bautiefe':str,
                    'Baugliedlänge':str,
                    'Nabensabstand':str,
                    'anderer Stuff':str,
                    'ZählernummerAlt':str,
                    'ZählerStandAlt':str,
                    'ZählernummerNeu':str}
    protokolle = glob.glob(pathPattern)
    #ToDo: Fehlerbehandlung wenn keine Dateien gefunden werden.
    protoDF = pd.DataFrame(columns=protoColumns.keys())
    for protokol in protokolle:
        protoWB = load_workbook(protokol)
        #Es wird vorrausgesetzt, dass das Worksheet "Dokumentation" heißt.
        try:
            protoWS = protoWB['Dokumentation']
        except:
            print("In der Datei "+protokol+" wurde kein Worksheet 'Dokumentation' gefunden!")
            break
        df = pd.read_excel(protokol, 
                           sheetname=0, 
                           names=protoColumns.keys(), 
                           dtype=protoColumns,  
                           parse_cols="D,E,F,G,H,I,J,L,R,V")
        df2appendHeizung = protoCleanupHeizung(df)
        df2appendHeizung['sourcefile'] = protokol
        protoDF = protoDF.append(df2appendHeizung)
        
    #Hier werden die Heizkostenverteilerdaten eingelesen
    protoColumns = {'ZählernummerAlt':str,
                    'ZählerStandAlt':str,
                    'ZählernummerNeu':str,
                    'ZählerStandNeu':str,}
    #ToDo: Fehlerbehandlung wenn keine Dateien gefunden werden.
    for protokol in protokolle:
        protoWB = load_workbook(protokol)
        #Es wird vorrausgesetzt, dass das Worksheet "Dokumentation" heißt.
        try:
            protoWS = protoWB['Dokumentation']
        except:
            print("In der Datei "+protokol+" wurde kein Worksheet 'Dokumentation' gefunden!")
            break
        df = pd.read_excel(protokol, 
                           sheetname=0, 
                           names=protoColumns.keys(), 
                           dtype=protoColumns,  
                           parse_cols="F,K,L,W")
        df2appendWasser = protoCleanupWasser(df)
        protoDF = protoDF.append(df2appendWasser)
    protoDF.reset_index(drop=True, inplace=True)
    return protoDF

def filterHK(df, value=None, dimension=None, deviation=0):
    try:
        return df[(value-(deviation*value) <= df[dimension]) & (df[dimension] <= value+(deviation*value))]
    except:
        print("Critical Error Processing: "+str(value)+", "+dimension)

def filterHKFactors(HKdf, laenge, hoehe, tiefe, deviation_threshold = 5):
    if type(laenge) is int:
        for deviation in range(0, deviation_threshold):
            HKtempdf_laenge = filterHK(HKdf, laenge, 'Länge', deviation*0.01)
            if (len(HKtempdf_laenge) > 0):
                print('Abweichung fuer Länge: '+str(deviation*0.01)+'%')
                break
    else:
        HKtempdf_laenge = HKdf
    #print(HKtempdf_laenge)
    if type(hoehe) is int:
        for deviation in range(0, deviation_threshold):
            HKtempdf_hoehe = filterHK(HKtempdf_laenge, hoehe, 'Höhe', deviation*0.01)
            if (len(HKtempdf_hoehe) > 0):
                print('Abweichung fuer Höhe: '+str(deviation*0.01)+'%')
                break
    else:
        HKtempdf_hoehe = HKtempdf_laenge
    #print(HKtempdf_hoehe)
    if type(tiefe) is int:
        for deviation in range(0, deviation_threshold):
            HKtempdf_tiefe = filterHK(HKtempdf_hoehe, tiefe, 'Tiefe', deviation*0.01)
            if (len(HKtempdf_tiefe) > 0):
                print('Abweichung fuer Tiefe: '+str(deviation*0.01)+'%')
                break
    else:
        HKtempdf_tiefe = HKtempdf_hoehe
    #print(HKtempdf_tiefe)       
    return HKtempdf_tiefe

def HKFactor(laenge, hoehe, tiefe, hkArt):
    HKFactors = HKdf
    #HKFactors = filterHKFactors(HKdf, laenge, hoehe, tiefe)
    #Enforce exact matching
    #print(str(laenge)+':'+str(type(laenge))+' '+str(hoehe)+':'+str(type(hoehe))+' '+str(tiefe)+':'+str(type(tiefe)))
    HKFactors = HKFactors[(HKFactors['Baulänge'] == laenge) &
                          (HKFactors['Bauhöhe'] == hoehe) &
                          (HKFactors['Bautiefe'] == tiefe) &
                          (HKFactors['HK-Art'] == hkArt)]
    if len(HKFactors) == 0:
        #print('-1: No Match')
        return -1
    elif len(HKFactors) > 1:
        print('-2: Multiple Matches')
        print(HKFactors)
        return -2
    #print("0: Exact Match")
    return float(HKFactors['75 / 65'].values[0])

In [ ]:
#Load Data

# Reads FactorData from file
HKdf = pd.read_excel(HeizkoerperfaktordatenPath, header=0)
HKdf = HKdf[["Baulänge","Bauhöhe","Bautiefe","HK-Art","75 / 65"]]
#HKdf.dropna(subset=['EN 75/65'], inplace=True)
HKdf.drop_duplicates(keep='first', inplace=True)
#HKdf[['Länge', 'Höhe', 'Tiefe']] = HKdf[['Länge', 'Höhe', 'Tiefe']].apply(pd.to_numeric)

#Read Protokolle
protoAll = collectProtocols(protoPathPattern)

In [ ]:
#Nimmt an, dass das Excel-sheet IMMER 'Zählerliste' heißt. Tippfehler gehen schief!!!
df = pd.read_excel(AuftragPath, sheetname='Zählerliste', header=9)
#Es wird vorrausgesetzt, das die Zaehlernummern immer in der 5.(E) Spalte stehen.
for Zählernummer in df['alte\nZähler-Nr']:
    #Fehlerbehandlung, wenn Feldinhalt keine Zählernummer
    try:
        proto = findZählernummer(Zählernummer, protoAll)
        if (len(proto['ZählernummerNeu'].values) > 0):
            df.loc[(df['alte\nZähler-Nr'] == Zählernummer),'neue\nZähler-Nr'] = proto['ZählernummerNeu'].values[0]
            #ToDo: Fehlerbehandlung wenn ein gültiger Zählerstand vorhanden.
            df.loc[(df['alte\nZähler-Nr'] == Zählernummer),'Stand\nAusbau'] = proto['ZählerStandAlt'].values[0]
            df.loc[(df['alte\nZähler-Nr'] == Zählernummer),'Stand\nEinbau'] = proto['ZählerStandNeu'].values[0]
            laenge = proto['Baulänge'].values[0]
            hoehe = proto['Bauhöhe'].values[0]
            tiefe = proto['Bautiefe'].values[0]
            hkArt = proto['HK-Art'].values[0]
        else:
            #print('Zählernummer '+str(Zählernummer)+' nicht gefunden')
            pass
    except:
        print('Bei der Bearbeitung von Zählernummer '+str(Zählernummer)+' ist ein Fehler aufgetreten')
    Faktor = HKFactor(laenge=laenge, hoehe=hoehe, tiefe=tiefe, hkArt=hkArt)
    if Faktor == -1:
        if ((str(laenge) == 'nan') & (str(hoehe) == 'nan') & (str(tiefe) == 'nan')):
            pass
        else:
            print('-1: Something bad happend: Zählernummer '+str(Zählernummer))
    elif not(type(Faktor) == float):
        print(str(Faktor))
    df.loc[(df['alte\nZähler-Nr'] == Zählernummer),'Faktor'] = Faktor

In [ ]:
HKbest = protoAll[['Bauhöhe',
          'Baulänge',
          'Bautiefe',
          'HK-Art',
          'Baugliedlänge',
          'Nabensabstand',
          'anderer Stuff']].dropna(subset=['Bauhöhe','Baulänge','Bautiefe','HK-Art']).drop_duplicates(keep='first')
HKbest["EN 75/65"] = 0

In [ ]:
AuftragOutput = './absys_files/A-2017-0060_919B_Output.xlsx'
writer = pd.ExcelWriter(AuftragOutput)
df.to_excel(writer, 'Zählerliste', index=False, startrow=9) 
writer.save()

In [ ]:
pathPattern='./absys_files/SenftenbergerRing*'
glob.glob(pathPattern)

In [ ]:
#This function exports the HKs, that need to be classified.
HKbest = protoAll[['Bauhöhe',
          'Baulänge',
          'Bautiefe',
          'HK-Art',
          'Baugliedlänge',
          'Nabensabstand',
          'anderer Stuff']].dropna(subset=['Bauhöhe','Baulänge','Bautiefe','HK-Art']).drop_duplicates(keep='first')
HKbest["EN 75/65"] = 0
writer = pd.ExcelWriter(HeizkoerperfaktordatenPath)
HKbest.to_excel(writer, 'HK', index=False)
writer.save()